# Exploring Temporal GNN Embeddings for Darknet Traffic Analysis
## Preprocessing Stages
___

## Table of Contents
1. [Filter Traces](#filter_traces) 
2. [Temporal GNN Preprocessing](#temporal_gnn_preprocessing)
3. [NLP preprocessing](#nlp_preprocessing)

In [1]:
from glob import glob 
import pandas as pd
import json
import sys
sys.path.append('../')

from src.preprocessing import *

# Total snapshots of the collection
TOT_DAYS = 31 
# Drop source hosts sending less than FILTER packets per snapshot
FILTER = 5 
# Generate the corpora keeping the top TOP_PORTS daily ports +1 as languages
TOP_PORTS = 2500 

## 1. Filter Traces

This code block represents a processing loop that iterates over different days' data. Each iteration follows these steps:

1. **Loading Raw Traces:** Data for a specific day is loaded into a dataframe. The day's information is used to indicate progress.

2. **Concatenate Dataframes:** The individual dataframes obtained for the day are combined into a single dataframe called `df`.

3. **Packets Filtering:** The `apply_packets_filter` function is used to filter packets in the `df` dataframe based on a specified filter called `FILTER`. This step narrows down the data based on specific packet properties.

4. **Ports Filtering:** The `apply_port_filter` function is applied to further refine the `df` dataframe based on the specified top ports `TOP_PORTS`. This step is especially useful when focusing on specific network port activity.

5. **Appending Processed Data:** The processed dataframe `df` is appended to the `processed_df` list, creating a collection of dataframes for each day.

6. **Saving Raw Dataset:** The `df` dataframe, containing processed data for the day, is saved to a CSV file named after the day. This step ensures that the processed data is stored for further analysis.

7. **Finalizing the Iteration:** The iteration for the current day is complete, and the loop advances to the next day, if available.


In [2]:
from tqdm.notebook import tqdm_notebook as tqdm

# Load Ground Truth and get the raw trace files list
flist = glob(f'../data/traces/*.log.gz')

# Initialize progress bar
pbar = tqdm(total=len(flist)+TOT_DAYS, desc='Setting up')

processed_df = []
for i in range(TOT_DAYS):
    dfs = []
    for file in sorted(flist)[24*i:24*(i+1)]:
        # Retrieve current day traces
        day = file.split('trace_')[-1][:8]
        df = load_single_file(file, day)
        dfs.append(df)
        
        # Update progress bar
        pbar.set_description(f'{day} Loading raw traces')
        pbar.update(1)
        
    # Get daily dataframe
    df = pd.concat(dfs, axis=0, ignore_index=True)
    
    # Packets filter
    pbar.set_description(f'{day} Running packets filter')
    df = apply_packets_filter(df, FILTER)
    
    # Ports filter
    pbar.set_description(f'{day} Running port filter')
    df = apply_port_filter(df, TOP_PORTS)
    
    processed_df.append(df)
    
    pbar.set_description(f'{day} Saving "raw" dataset')
    df.to_csv(f'../data/raw/raw_{day}.csv', index=False)
    
    # Update progress bar
    pbar.update(1)

Setting up:   0%|          | 0/775 [00:00<?, ?it/s]

## 2. Temporal GNN preprocessing

### Loading Filtered Traces and Ground Truth

This code snippet focuses on loading and concatenating processed trace data. The following steps outline the process:

1. **Loading Ground Truth Data:** The ground truth data is loaded from a CSV file located at `../data/ground_truth/ground_truth.csv`. This data is essential for comparison and evaluation purposes.

2. **Loading Preprocessed Traces:** An empty list `raw_df` is initialized to store individual dataframes for each trace file. A loop iterates over a list of trace files found in the `../data/raw/` directory.

3. **Loading and Appending:** Within the loop, each trace file is loaded as a dataframe using the `pd.read_csv` function. The loaded dataframe is appended to the `raw_df` list.

4. **Combining Dataframes:** After loading all trace files, the individual dataframes stored in the `raw_df` list are combined into a single dataframe using the `pd.concat` function. The `ignore_index=True` parameter ensures that the index is reset in the final combined dataframe.

5. **Finalizing Processed Data:** The combined dataframe, named `raw_df`, now contains the processed trace data from all the loaded files. It is ready for further analysis and evaluation.

In [3]:
from tqdm.notebook import tqdm_notebook as tqdm
from src.preprocessing.gnn import *

# Load ground truth
gt = pd.read_csv(f'../data/ground_truth/ground_truth.csv')

# Load preprocessed traces
raw_df = []
for file in tqdm(sorted(glob(f'../data/raw/*')), desc='Loading filtered traces'):
    raw_df.append(pd.read_csv(file))

# Concatenate the loaded dataframes
raw_df = pd.concat(raw_df, ignore_index=True)

Loading filtered traces:   0%|          | 0/31 [00:00<?, ?it/s]

### Generate graph

This code segment involves the aggregation of edges and the generation of snapshots. It follows these steps:

1. **Aggregating Edges and Merging Ground Truth:** Edges are aggregated, and the ground truth data is merged with the aggregated edges. The `dst_port` column in the dataframe is converted to string type. This process prepares the data for further manipulation.

2. **Getting Distinct Nodes:** Unique source IP addresses (`src_ip`) and destination port numbers (`dst_port`) are extracted from the dataframe. These distinct nodes serve as identifiers for network nodes in the analysis.

3. **Building Lookup Dictionaries:** Lookup dictionaries are constructed to map unique source IP addresses and destination port numbers to numerical identifiers. This mapping is essential for node representation in the analysis.

4. **Converting Nodes to IDs:** The IP addresses and port numbers in the dataframe are replaced with their corresponding numerical identifiers using the lookup dictionaries. This conversion prepares the data for generating snapshots.

5. **Generating Snapshots:** The dataframe is processed in snapshots, each corresponding to a specific time interval (`day`). For each interval, a snapshot is extracted, containing the relevant network data.

6. **Saving Snapshots:** Each generated snapshot is saved as a text file in the `../data/graph/` directory. These snapshot files capture the network state at different intervals.

7. **Saving Lookup Dictionaries:** The lookup dictionaries for IP addresses and port numbers are saved as JSON files in the `../data/graph/` directory. These dictionaries provide the mapping between numerical identifiers and their corresponding network nodes.

In [4]:
from tqdm.notebook import tqdm_notebook as tqdm

# Initialize progress bar
pbar = tqdm(total=4, desc='Agregating edges')

# Aggregate edges and merge gt
df = aggregate_edges(raw_df, gt)
df['dst_port'] = df['dst_port'].astype(str)
pbar.update(1)

# Get distinct nodes
pbar.set_description(f'Getting distinct nodes')
unique_ips = df.src_ip.unique()
unique_ports = df.dst_port.unique()
pbar.update(1)

# Build lookup dictionaries
pbar.set_description(f'Building lookup dictionaries')
ip_lookup = {ip:i for i,ip in enumerate(unique_ips)}
port_lookup = {p:i+len(ip_lookup) for i,p in enumerate(unique_ports)}
pbar.update(1)

# Convert nodes to node IDs
pbar.set_description(f'Converting nodes to IDs')
df['src_ip'] = df['src_ip'].map(lambda x: ip_lookup[x])
df['dst_port'] = df['dst_port'].map(lambda x: port_lookup[x])
pbar.update(1)
pbar.close()

# Process each snapshot
intervals = sorted(df.interval.unique())
for day in tqdm(intervals, desc='Generating snapshots'):
    # Extract current snapshot
    snapshot = extract_single_snapshot(df, day)

    # Save current snapshot
    fname = f'../data/graph/{day}.txt'
    with open(fname, 'w') as file:
        file.write(snapshot)
        
# Save Host node lookup dictionary
with open(f'../data/graph/ip_lookup.json', 'w') as file:
    json.dump(ip_lookup, file)
# Save Port node lookup dictionary
with open(f'../data/graph/port_lookup.json', 'w') as file:
    json.dump(port_lookup, file)

Agregating edges:   0%|          | 0/4 [00:00<?, ?it/s]

Generating snapshots:   0%|          | 0/31 [00:00<?, ?it/s]

### Extract node features

This code snippet focuses on feature extraction and data saving. The following steps outline the process:

1. **Loading Lookup Dictionaries:** The lookup dictionaries for IP addresses and port numbers are loaded from JSON files. These dictionaries contain mappings between numerical identifiers and network nodes.

2. **Iterating Over Time Intervals:** The code iterates over the total intervals found in the `raw_df` dataframe, each corresponding to a specific time interval (`day`).

3. **Extracting Current Snapshot:** A snapshot is extracted from the `raw_df` dataframe for the current time interval (`day`).

4. **Extracting Node Features (IP Layer):** A series of functions are applied to extract various node features related to the IP layer. These functions include obtaining contacted destination ports, statistics per destination port, contacted destination IP addresses, statistics per destination IP address, and packet statistics based on source IP addresses. These features provide insights into the network activity at the IP level.

5. **Uniform Features with IP Lookup:** The extracted IP-based node features are uniformized using the `ip_lookup` dictionary to map IP addresses to numerical identifiers.

6. **Extracting Node Features (Port Layer):** Similar to the IP layer, node features for the port layer are extracted. Functions are applied to gather information about contacted source IP addresses, statistics per source IP address, contacted destination IP addresses (dummy values), statistics per destination IP address (dummy values), and packet statistics based on destination port numbers.

7. **Uniform Features with Port Lookup:** The extracted port-based node features are uniformized using the `port_lookup` dictionary to map port numbers to numerical identifiers.

8. **Concatenating Node Features:** The node features extracted from both IP and port layers are concatenated into a single dataframe named `features`. This dataframe captures the network characteristics for the current time interval (`day`).

9. **Sorting and Saving Features:** The `features` dataframe is sorted by index and saved as a CSV file. The saved file captures the feature data for the specific time interval and layer. The destination path for the saved file is determined based on the value of `DSET_TYPE`.

In [7]:
from tqdm.notebook import tqdm_notebook as tqdm

# Load saved lookup dictionary for Host node layer
with open(f'../data/graph/ip_lookup.json', 'r') as file:
    ip_lookup = json.loads(file.read())
# Load saved lookup dictionary for Port node layer
with open(f'../data/graph/port_lookup.json', 'r') as file:
    port_lookup = json.loads(file.read())

tot_intervals = sorted(raw_df.interval.unique())
for day in tqdm(tot_intervals):
    # Extract current snapshot
    snapshot = raw_df[raw_df.interval==day]
    
    # Extract node features -- Host layer
    node_ip_features = [
        get_contacted_dst_ports(snapshot),
        get_stats_per_dst_port(snapshot),
        get_contacted_dst_ips(snapshot),
        get_stats_per_dst_ip(snapshot),
        get_packet_statistics(snapshot, by='src_ip')
    ]
    node_ip_features = uniform_features(
        node_ip_features, ip_lookup, 'src_ip'
    )

    # Extract node features -- Port layer
    node_port_features = [
        get_contacted_src_ips(snapshot),
        get_stats_per_src_ip(snapshot),
        get_contacted_dst_ips(snapshot, dummy=True),
        get_stats_per_dst_ip(snapshot, dummy=True),
        get_packet_statistics(snapshot, by='dst_port')
    ]
    node_port_features = uniform_features(
        node_port_features, port_lookup, 'dst_port'
    )
    
    # Concatenate the node features and save
    features = pd.concat([node_ip_features, node_port_features]).sort_index()
    features.to_csv(f'../data/features/features_{day}.csv')

  0%|          | 0/31 [00:00<?, ?it/s]

## 3. NLP preprocessing

The following codes start from preprocessed filtered traces and generate textual corpora which can be processed by NLP algorithms.

### Loading Filtered Traces

This code segment focuses on loading preprocessed traces and initializing data. The following steps outline the process:

1. **Loading Preprocessed Traces:** The code checks if the `raw_df` variable is already initialized. If not, it initializes an empty list named `raw_df`. The code then iterates over a list of trace files found in the `../data/raw/` directory.

2. **Loading and Concatenating:** Within the loop, each trace file is loaded as a dataframe using the `pd.read_csv` function. The loaded dataframes are appended to the `raw_df` list.

3. **Combining Dataframes:** After loading all trace files, the individual dataframes stored in the `raw_df` list are combined into a single dataframe using the `pd.concat` function. The `ignore_index=True` parameter ensures that the index is reset in the final combined dataframe.

4. **Data Initialization:** If the `raw_df` variable was previously uninitialized, it is now populated with the concatenated dataframe containing preprocessed trace data. This dataframe will serve as the basis for further analysis.

In [8]:
from tqdm.notebook import tqdm_notebook as tqdm
from src.preprocessing.nlp import *
import pickle

# Load preprocessed traces
if type(raw_df) == type(None):
    raw_df = []
    for file in tqdm(sorted(glob(f'../data/raw/*'))):
        raw_df.append(pd.read_csv(file))
    
    # Concatenate dataframes
    raw_df = pd.concat(raw_df, ignore_index=True)

### Corpus generation

This code segment focuses on generating a corpus and saving it to disk. The following steps outline the process:

1. **Iterating Over Time Intervals:** The code iterates over the total intervals found in the `raw_df` dataframe, each corresponding to a specific time interval (`day`).

2. **Extracting Current Snapshot and Sorting:** A snapshot is extracted from the `raw_df` dataframe for the current time interval (`day`). The snapshot is sorted by the `ts` column in ascending order.

3. **Generating Corpus by Port:** A corpus is generated based on the `src_ip` values grouped by `dst_port`. The list of `dst_port` values is ordered based on the frequency of occurrence. The resulting corpus structure contains a list of `src_ip` values for each `dst_port`.

4. **Moving "Other" Port to Bottom:** The `src_ip` lists for the top 2500 `dst_port` values are reordered. The remaining ones are associated with the "other" port and moved to the bottom of the corpus.

5. **Flattening the Corpus:** The corpus is flattened into a single array of `src_ip` values across all `dst_port` values.

6. **Removing Duplicates and Splitting:** Duplicate `src_ip` values are removed from the corpus. The corpus is then split into equally sized chunks, each containing a maximum of 1000 `src_ip` values.

7. **Converting to List Format:** The corpus chunks are converted to list format for compatibility with pickle.

8. **Saving the Corpus:** The generated corpus is saved as a pickle file named after the specific time interval (`day`). The pickle file is stored in the `../data/corpus/` directory.

In [10]:
from tqdm.notebook import tqdm_notebook as tqdm

tot_intervals = sorted(raw_df.interval.unique())
for day in tqdm(tot_intervals):
    # Extract current snapshot
    snapshot = raw_df[raw_df.interval==day].sort_values('ts')
    
    # Get the list of src IP by port
    port_order = snapshot.value_counts('dst_port').index
    corpus = snapshot.groupby('dst_port').agg({'src_ip':list}).reindex(port_order)

    # Move port "other" at bottom
    corpus = pd.concat([corpus.iloc[1:], corpus.iloc[:1]], ignore_index=True)
    corpus = np.hstack([x for x in corpus.src_ip])

    # Remove duplicates and split the corpus in equally sized chunks
    corpus = drop_duplicates(corpus)
    corpus = split_array(corpus, step=1000)
    corpus = [list(x) for x in corpus]

    # Save the corpus
    with open(f'../data/corpus/corpus_{day}.pkl', 'wb') as file:
        pickle.dump(corpus, file)

  0%|          | 0/31 [00:00<?, ?it/s]